# Ejemplos para átomos monoelectrónicos
Autores: Alejandro Villar y Jaime Rosado

### Comandos para ejecutar este cuaderno desde Google Colab

Monta la unidad de Google Drive asociada a tu **cuenta UCM**. Hay que dar permisos a Colab para que acceda al Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Sigue este enlace para tener acceso a una carpeta compartida llamada 'ProgramasFAyM', la cual contiene los módulos donde está el código que se usa en este cuaderno.

https://drive.google.com/drive/folders/1cSm1S8fYvurHPbSvcQqpm7sluvI1Ez9-?usp=sharing

A continuación, **genera un acceso directo a esta carpeta** en el directorio principal de tu unidad de Google Drive (¡ojo, a la carpeta, no a los ficheros que contiene!). Para ello, usa la opción correspondiente del menú desplegable que aparece al pinchar la carpeta con el botón derecho del ratón.

Añade la ruta de esta carpeta compartida a la ruta de búsqueda de Python.

In [ ]:
import sys
Mpath = '/content/drive/MyDrive/ProgramasFAyM'
#Mpath = '/content/drive/My Drive/ProgramasFAyM'
#Mpath = '/drive/MyDrive/ProgramasFAyM'
#Mpath = '/drive/My Drive/ProgramasFAyM'
sys.path.append(Mpath)

Importa el módulo donde se encuentra el código del programa y la librería numpy que es necesaria para algunos cálculos. Sólo es necesario hacerlo una vez, aunque el programa se ejecute varias veces.

Si no encuentra el módulo puede significar que no se ha creado correctamente el acceso directo a la carpeta compartida o que la ruta de esta carpeta es diferente a lo indicado en la celda anterior. En este último caso, prueba alguna de las alternativas de `Mpath` quitando el símbolo `#` que tiene delante. Si esto sigue sin funcionar, abre el árbol de archivos (icono con forma de carpeta a la izquierda de esta ventana) del entorno de ejecución y busca el acceso directo a la carpeta compartida en tu unidad Drive. Al pinchar con el botón derecho sobre ella, aparece un menú desplegable con la opción `copiar ruta`, que permite hallar la ruta correcta para dar a la variable `Mpath`.

In [ ]:
import atomictools as at
import numpy as np

En la misma carpeta compartida donde está el código del programa, se han subido unos ficheros con las soluciones numéricas a la ecuación de Schrödinger radial con un modelo de potencial central para el átomo de carbono. Hay un fichero para cada nivel ocupado del átomo en su configuración fundamental 1s$^2$ 2s$^2$ 2p$^2$.

A continuación pasa la ruta de estos ficheros a la función `R_num` para cargar la función radial en este entorno de trabajo.

In [ ]:
C_1s_num = at.R_num(Mpath + '/files/C_1s.txt')
C_2s_num = at.R_num(Mpath + '/files/C_2s.txt')
C_2p_num = at.R_num(Mpath + '/files/C_2p.txt')

Vamos a comparar estas soluciones numéricas con las soluciones hidrogenoides para un potencial coulombiano de carga $Z=6$.

Obtenemos primero las funciones radiales hidrogenoides con la función `R_hydog`.

In [ ]:
C_1s_Z = at.R_hydrog(n=1, l=0, Z=6)
C_2s_Z = at.R_hydrog(n=2, l=0, Z=6)
C_2p_Z = at.R_hydrog(n=2, l=1, Z=6)

A continuación, comparamos gráficamente los resultados. Para el nivel 1s, no se aprecian grandes diferencias entre la solución numérica y la solución hidrogenoide (nótese que la escala horizontal es distinta).
Para los otros niveles sí se aprecian diferencias claras en las posiciones de los nodos y los máximos de probabilidad, sobre todo para el nivel 2p, donde el desplazamiento del máximo es muy notable. Esto se debe al apantallamiento que producen los electrones más internos.

In [ ]:
#C_1s_num.plot_P()
#C_2s_num.plot_P()
C_2p_num.plot_P()

#C_1s_Z.plot_P()
#C_2s_Z.plot_P()
C_2p_Z.plot_P()

Podemos construir orbitales con la función `orbital` a partir de las funciones radiales numéricas anteriores y los armónicos esféricos correspondientes (o funciones angulares radiales).

In [ ]:
Y00 = at.spherical_harmonic(l=0, m=0)
Y10 = at.spherical_harmonic(l=1, m=0)
Y11 = at.spherical_harmonic(l=1, m=1)
Y1_1 = at.spherical_harmonic(l=1, m=-1)

orb_1s = at.orbital(f_rad=C_1s_num, f_ang=Y00)
orb_2s = at.orbital(f_rad=C_2s_num, f_ang=Y00)
orb_2p0 = at.orbital(f_rad=C_2p_num, f_ang=Y10)
orb_2p1 = at.orbital(f_rad=C_2p_num, f_ang=Y11)
orb_2p_1 = at.orbital(f_rad=C_2p_num, f_ang=Y1_1)

La visualización 3D es muy similar a la de un orbital del átomo de hidrógono. La principal diferencia es el tamaño del orbital.

In [ ]:
#orb_1s.plot_volume()
#orb_2s.plot_volume()
orb_2p0.plot_volume()
#orb_2p1.plot_volume()
#orb_2p_1.plot_volume()

No se puede visualizar la distribución de probabilidad de todos los electrones del átomo (es una distribución en $6\cdot 3=18$ dimensiones!!). Pero sí es fácil obtener la distribución espacial de carga. Ésta se obtiene sumando las distribuciones espaciales de probabilidad de todos los orbitales ocupados (sumando también sobre espines). Nótese que el resultado dependerá del estado en que se encuentre el átomo. Asumiendo que el estado se describe por un determinante de Slater, se obtiene el siguiente resultado.

In [ ]:
orb = at.orbital_hydrog(n=1, l=0, m=0) # Sólo es para generar un objeto de tipo orbital
# Niveles 1s2 2s2
prob.orb = 2. * orb_1s.prob #+ 2. * orb_2s.prob
# Determinante (0+ 0-)
#orb.prob += 2. * orb_2p0.prob
# Determinante (1+ 0+)
orb.prob += 1. * orb_2p1.prob + 1. * orb_2p0.prob
orb.plot_volume()

Para todos los estados se obtiene básicamente la misma figura porque la distribución del orbital 1s está mucho más concentrada que la de los orbitales apantallados 2s y 2p, que no se aprecian por la escala de colores del plot de volumen.

Si nos centramos ahora en los electrones de valencia 2p, la distribución de carga es la siguiente. Nótese el cambio de escala en la densidad de probabilidad de la barra de colores.

In [ ]:
orb = at.orbital_hydrog(n=1, l=0, m=0) # Sólo es para generar un objeto de tipo orbital
# Determinante (0+ 0-)
#orb.prob = 2. * orb_2p0.prob
# Determinante (1+ 0+)
orb.prob = 1. * orb_2p1.prob + 1. * orb_2p0.prob
orb.plot_volume()

Un último resultado interesante que se puede obtener en este ejemplo es demostrar que la distribución espacial de carga de un nivel completo tiene simetría esférica. Para ello, sumamos las distribuciones de probabilidad de los 3 orbitales 2p (6 si tenemos en cuenta las orientaciones de espín).

In [ ]:
orb = at.orbital_hydrog(n=1, l=0, m=0) # Sólo es para generar un objeto de tipo orbital
orb.prob = 2. * orb_2p0.prob + 2. * orb_2p1.prob + 2. * orb_2p_1.prob
orb.plot_volume()